In [1]:
import tensorflow as tf

from tensorflow import keras

In [2]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self,strides,size_1,size_2,size_3,size_4,filters_1,filters_2,filters_3,filters_4,activation='relu',**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            
            keras.layers.Conv2D(filters_1,size_1,strides=strides,padding='same',use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters_1,size_2,strides=1,padding='same',use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,

            keras.layers.Conv2D(filters_3,size_3,strides=1,padding='same',use_bias= False),
            keras.layers.BatchNormalization(),
            self.activation


        ]
        self.skip_layers = [
            
            keras.layers.Conv2D(filters_4,size_4,strides=strides,padding='same',use_bias = False),
            keras.layers.BatchNormalization(),
            self.activation
            
            
        ]
        
        
    def call(self,inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(skip_Z+Z)
            
        

In [3]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64,7,strides=2,input_shape=[224,224,3],padding='same',use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same'))

for i in range(3):

    model.add(ResidualUnit(strides = 1, size_1=1,size_2=3,size_3=1,size_4=1 ,filters_1=64,filters_2=64,filters_3=256,filters_4=256))

for i in range(8):
    
    if i == 0 : 

        model.add(ResidualUnit(strides = 2, size_1=1,size_2=3,size_3=1,size_4=1 ,filters_1=128,filters_2=128,filters_3=512,filters_4=512))

    else:
        
        model.add(ResidualUnit(strides = 1, size_1=1,size_2=3,size_3=1,size_4=1 ,filters_1=128,filters_2=128,filters_3=512,filters_4=512))

        
for i in range(36):
    
    if i == 0 : 

        model.add(ResidualUnit(strides = 2, size_1=1,size_2=3,size_3=1,size_4=1 ,filters_1=256,filters_2=256,filters_3=1024,filters_4=1024))

    else:
        
        model.add(ResidualUnit(strides = 1, size_1=1,size_2=3,size_3=1,size_4=1 ,filters_1=256,filters_2=256,filters_3=1024,filters_4=1024))
for i in range(3):
    
    if i == 0 : 

        model.add(ResidualUnit(strides = 2, size_1=1,size_2=3,size_3=1,size_4=1 ,filters_1=512,filters_2=512,filters_3=2048,filters_4=2048))

    else:
        model.add(ResidualUnit(strides = 1, size_1=1,size_2=3,size_3=1,size_4=1 ,filters_1=512,filters_2=512,filters_3=2048,filters_4=2048))

        
        
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1000,activation = 'softmax'))


2022-07-08 10:55:27.108516: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-08 10:55:27.109744: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [4]:
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 112, 112, 64)      9408      
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 64)      256       
_________________________________________________________________
activation (Activation)      (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 64)        0         
_________________________________________________________________
residual_unit (ResidualUnit) (None, 56, 56, 256)       76288     
_________________________________________________________________
residual_unit_1 (ResidualUni (None, 56, 56, 256)       137728    
_________________________________________________________________
residual_unit_2 (ResidualUni (None, 56, 56, 256)       1